In [ ]:
#import libraries
import json
from openai import OpenAI
import pandas as pd
from IPython.display import Image, display

import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [ ]:
#initialize OpenAI client
client = OpenAI()

In [ ]:
#import dataset
data_path = r"C:\Users\ASUS\Downloads\deep learning\beforelabeled.xlsx"
df = pd.read_excel(data_path)

In [ ]:
df.head()

,comment
0,did he come around
1,why did she smile
2,when i found my lost toy after 12 years later
3,hahahaha
4,she is an exceptional judge i love her respec...


In [ ]:
#prompt for sentiment analysis
sentiment_system_prompt = '''
Your goal is to analyze YouTube comments and label them with their sentiment as either "positive", "negative", or "neutral".
You will be provided with a YouTube comment, and you will output a json object containing the following information:

{
    sentiment: string // Sentiment label for the comment, which can only be "positive", "negative", or "neutral".
}
Ensure that your sentiment labels are based directly on the language and sentiment conveyed in the comment. Only use these three categories: "positive", "negative", and "neutral". Do not assign mixed or undefined labels.

'''

In [ ]:
#function to get sentiment of a comment
def get_sentiment(comment):
    completion = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature = 0.7,
    response_format={"type":"json_object"},
    messages=[
        {"role": "system", "content": sentiment_system_prompt},
        {
            "role": "user",
            "content": comment
        }
    ]
    )

    return completion.choices[0].message.content

In [ ]:
#cal token count
model = "gpt-4o-mini"
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
num_tokens_from_string(sentiment_system_prompt, model)

119

In [ ]:
#test
for _, row in df[:5].iterrows():
    comment = row["comment"]
    result = get_sentiment(comment)
    print(f"comment:{comment} \n sentiment: {result}")
    print("\n\n----------------------------\n\n")

In [ ]:
#split data into batches
tasks = []
for index, row in df.iterrows():
    comment = row["comment"]
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": model,
            "messages": [
                {
                    "role": "system",
                    "content": sentiment_system_prompt
                },
                {
                    "role": "user",
                    "content": comment
                }
            ]
        }
    }
    tasks.append(task)

batch_size = 10000
num_batches = (len(tasks) + batch_size - 1) // batch_size  # Calculate the number of batches

batches = []
for i in range(num_batches):
    batch_start = i * batch_size
    batch_end = min((i + 1) * batch_size, len(tasks))
    batch = tasks[batch_start:batch_end]
    batches.append(batch)

print(f"Total batches: {num_batches}")

Total batches: 8


In [ ]:
for i in range(num_batches):
    batch = batches[i]
    print(f"Batch {i + 1}: {len(batch)} tasks")

Batch 1: 10000 tasks
Batch 2: 10000 tasks
Batch 3: 10000 tasks
Batch 4: 10000 tasks
Batch 5: 10000 tasks
Batch 6: 10000 tasks
Batch 7: 10000 tasks
Batch 8: 7510 tasks


In [ ]:

#save the batch
batch_path = r"C:\Users\ASUS\Downloads\deep learning\data"
for i in range(num_batches):
    batch = batches[i]
    with open(rf"{batch_path}\batch_{i}.jsonl", "w") as f:
        for obj in batch:
            f.write(json.dumps(obj) + "\n")

In [ ]:
#upload batch file
batch_job = []
for i in range(num_batches):
    batch = batches[i]
    batch_file_path = rf"{batch_path}\batch_{i}.jsonl"
    batch_file = client.files.create(
        file=open(batch_file_path, "rb"),
        purpose="batch"
    )
    batch_job.append(batch_file)

NameError: name 'num_batches' is not defined

In [ ]:
#print batch job
for i in range(num_batches):
    print(f"Batch {i + 1}: {batch_job[i].id}")

NameError: name 'num_batches' is not defined

In [ ]:
#creating batch job
batch_job_crt = []
for i in range(num_batches):
    batch_job_crt_var = client.batches.create(
        input_file_id = batch_job[i].id,
        endpoint = "/v1/chat/completions",
        completion_window = "24h",
    )
    batch_job_crt.append(batch_job_crt_var)

In [ ]:
#check the status of the batch job
for i in range(num_batches):
    batch_job = client.batches.retrieve(batch_job_crt[i].id)
    print(f"Batch {i + 1} status: {batch_job.status}")
    if batch_job.status == "failed":
        print(batch_job)

Batch 1 status: in_progress
Batch 2 status: failed
Batch(id='batch_675b9ad36cf481909eb68d86787092ee', completion_window='24h', created_at=1734056659, endpoint='/v1/chat/completions', input_file_id='file-4CfEumPwu4EcdzsvJfGPXT', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4o-mini in organization org-KpaeIqASYmq2rTRwJWb5i0Da. Limit: 2,000,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list'), expired_at=None, expires_at=1734143059, failed_at=1734056662, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
Batch 3 status: failed
Batch(id='batch_675b9ad423fc8190b7af2dc28b225d7a', completion_window='24h', created_at=1734056660, endpoint='/v1/chat/completio

In [ ]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [ ]:
with open(rf"{batch_path}\batch_{i}_result.jsonl", 'wb') as file:
    file.write(result)

In [ ]:
# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [ ]:
#test
for res in results[:100]:
    task_id = res['custom_id']
    index = task_id.split('-')[-1]
    result = res['response']['body']['choices'][0]['message']['content']
    comment = df.iloc[int(index)]
    comment = comment['comment']
    print(f"comment:{comment} \n sentiment: {result}")
    print("\n\n----------------------------\n\n")

comment:did he come around  
 sentiment: {
    "sentiment": "neutral"
}


----------------------------


comment:why did she smile   
 sentiment: {
    "sentiment": "neutral"
}


----------------------------


comment:when i found my lost toy after 12 years later 
 sentiment: {
    "sentiment": "positive"
}


----------------------------


comment:hahahaha 
 sentiment: {
    "sentiment": "positive"
}


----------------------------


comment:she is an exceptional judge  i love her respect to other people  she has a good heart reminds me of judge frank caprio who i simply adore and love they are a perfect example to other judges  
 sentiment: {
    "sentiment": "positive"
}


----------------------------


comment:the coolest kid in middle school        
 sentiment: {
    "sentiment": "positive"
}


----------------------------


comment:maybe he needed that reminder   that he was the nicest kid in the middle school  
 sentiment: {
    "sentiment": "positive"
}


------------------------